In [1]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Analyzing a GEOGRAPHY column with `bigframes.geopandas.GeoSeries`

In [2]:
import bigframes
import bigframes.geopandas
import bigframes.pandas as bpd
bpd.options.display.progress_bar = None

### 1. Load the Counties table from the Census Bureau US Boundaries dataset

In [3]:
df = bpd.read_gbq("bigquery-public-data.geo_us_boundaries.counties")

/usr/local/google/home/arwas/src/bigframes3/bigframes/session/_io/bigquery/read_gbq_table.py:280: DefaultIndexWarning: Table 'bigquery-public-data.geo_us_boundaries.counties' is clustered and/or partitioned, but BigQuery DataFrames was not able to find a suitable index. To avoid this warning, set at least one of: `index_col` or `filters`.
  warnings.warn(msg, category=bfe.DefaultIndexWarning)


### 2. Create a series from the int_point_geom column

In [4]:
point_geom_series = df['int_point_geom']

## The `GeoSeries` constructor accepts local data or a `bigframes.pandas.Series` object.

### 1. Create a GeoSeries from local data with `Peek`

In [5]:
five_geo_points = point_geom_series.peek(n = 5)
five_geo_points

171     POINT (-95.50742 42.39186)
219    POINT (-105.42894 37.27755)
402     POINT (-93.34905 32.10121)
526     POINT (-84.60469 43.29233)
677      POINT (-89.5681 37.04779)
Name: int_point_geom, dtype: geometry

### Convert the five geo points to `bigframes.gopandas.GeoSeries`

In [6]:
geo_points = bigframes.geopandas.GeoSeries(
        [point for point in five_geo_points]
)
geo_points

0     POINT (-95.50742 42.39186)
1    POINT (-105.42894 37.27755)
2     POINT (-93.34905 32.10121)
3     POINT (-84.60469 43.29233)
4      POINT (-89.5681 37.04779)
dtype: geometry

### Retrieve the x (longitude) and y (latitude) from the GeoSeries with `.x` and `.y`.

#### Note: TypeError is raised if `.x` and `.y` are used with a geometry type other than `Point`.

### `.x`

In [7]:
geo_points.x

0   -95.507421
1   -105.42894
2    -93.34905
3    -84.60469
4   -89.568097
dtype: Float64

### `.y`

In [8]:
geo_points.y

0     42.39186
1    37.277547
2    32.101213
3    43.292326
4    37.047793
dtype: Float64

### 2. Alternatively, use the `.geo` accessor to access GeoSeries methods from a `bigframes.pandas.Series` object.

#### `geo.x`

In [9]:
point_geom_series.geo.x

0    -101.298265
1     -99.111085
2      -66.58687
3    -102.601791
4     -71.578625
5     -88.961529
6     -87.492986
7     -82.422666
8    -100.208166
9     -85.815939
10   -101.681133
11   -119.516659
12    -89.398306
13    -107.78848
14    -91.159306
15   -113.887042
16    -83.470416
17    -98.520146
18    -83.911718
19    -87.321865
20    -91.727626
21    -93.466093
22   -101.143324
23    -78.657634
24    -94.272323
dtype: Float64

#### `geo.y`

In [10]:
point_geom_series.geo.y

0     46.710819
1     29.353661
2     18.211152
3     38.835646
4     41.869768
5     39.860237
6     36.892059
7     38.143642
8     34.524623
9     30.862007
10    40.180165
11    46.228125
12    36.054196
13    38.154731
14    38.761902
15    44.928506
16    30.447232
17    29.448671
18    42.602532
19    34.529776
20    33.957675
21    42.037538
22    29.875285
23    36.299884
24    44.821657
dtype: Float64

## Retrive the `area` of different geometry shapes. 

### 1. Create a geometry collection from local data with `Peek`

In [11]:
geom_series = df["county_geom"].peek(n = 5)
geom_series

54     POLYGON ((-93.76575 45.06448, -93.76575 45.064...
256    POLYGON ((-89.83723 42.68318, -89.83732 42.682...
266    POLYGON ((-104.19381 39.56523, -104.19464 39.5...
485    MULTIPOLYGON (((-91.05884 32.17233, -91.05891 ...
765    POLYGON ((-83.61848 38.1557, -83.61861 38.1554...
Name: county_geom, dtype: geometry

### 2. Convert the geometry collection to `bigframes.gopandas.GeoSeries`

In [12]:
five_geom = bigframes.geopandas.GeoSeries(
        [point for point in geom_series]
)
five_geom

0    POLYGON ((-93.76575 45.06448, -93.76575 45.064...
1    POLYGON ((-89.83723 42.68318, -89.83732 42.682...
2    POLYGON ((-104.19381 39.56523, -104.19464 39.5...
3    MULTIPOLYGON (((-91.05884 32.17233, -91.05891 ...
4    POLYGON ((-83.61848 38.1557, -83.61861 38.1554...
dtype: geometry

## Note: `bigframes.geopandas.GeoSeries.area` raises NotImplementedError.  

In [13]:
five_geom.area

NotImplementedError: GeoSeries.area is not supported. Use bigframes.bigquery.st_area(series), instead. Share your usecase with the BigQuery DataFrames team at the https://bit.ly/bigframes-feedback survey.You are currently running BigFrames version 1.35.0

### 3. Use `bigframes.bigquery.st_area` to retirive the `area` in square meters instead. See: https://cloud.google.com/bigquery/docs/reference/standard-sql/geography_functions#st_area

In [14]:
import bigframes.bigquery as bbq

In [15]:
geom_area = bbq.st_area(five_geom)
geom_area

0    1567505274.453911
1    1511436852.079554
2    4789800692.948824
3    1686877416.586061
4     740944862.916908
dtype: Float64

##  Use `bigframes.geopandas.GeoSeries.from_xy()` to create a GeoSeries of `Point` geometries. 

### 1. Reuse the `geo_points.x` and `geo_points.y` results by passing them to `.from_xy()` 

In [16]:
bigframes.geopandas.GeoSeries.from_xy(geo_points.x, geo_points.y)

0     POINT (-95.50742 42.39186)
1    POINT (-105.42894 37.27755)
2     POINT (-93.34905 32.10121)
3     POINT (-84.60469 43.29233)
4      POINT (-89.5681 37.04779)
dtype: geometry